In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [11]:
# hyperparameters
seed = 42
valid_size=0.3 # validation + testing
test_size=0.5 # tesing / (testing+validation)
learning_rate = 0.001
epochs = 10
batch_size=32

random.seed(seed)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device=torch.device('cpu')
print(device)

cuda


In [5]:
# load data
df_abstract = pd.read_csv('../processed_data/df_abstract.csv').to(device)
df_cascade = pd.read_csv('../processed_data/df_cascade.csv').to(device)
df_authors = pd.read_csv('../processed_data/df_authors.csv').to(device)
train_cascade, temp_cascade = train_test_split(df_cascade, test_size=valid_size, random_state=seed)
val_cascade, test_cascade = train_test_split(temp_cascade, test_size=test_size, random_state=seed)

df_cascade['citation_authors_length'] = df_cascade['citation_authors'].str.split(', ').str.len()
length = df_cascade['citation_authors_length'].sum()
avg_negative = length//len(df_cascade)

In [52]:
# add negative samples
author_ls = df_authors['author_id'].tolist()
for index, row in df_cascade.iterrows():
    authors = [int(x) for x in row['citation_authors'].split(', ')]
    temp_indices = []
    for _ in range(avg_negative):
        isInclude = True
        while(isInclude):
            rand_author_id = random.choice(author_ls)
            if rand_author_id in authors:
                continue
            else:
                isInclude = False
                temp_indices.append(rand_author_id)
    df_cascade.at[index, 'negative_authors'] = ', '.join(str(x) for x in temp_indices)


In [57]:
# split
train_data, temp_data = train_test_split(df_cascade, test_size=valid_size, random_state=seed)
val_data, test_data = train_test_split(temp_data, test_size=test_size, random_state=seed)

train_iter = []
for index, row in train_data.iterrows():
    paper_id = row['paper_id']
    for author in row['citation_authors'].split(', '):
        train_iter.append([paper_id, int(author), 1])
    for negative in row['negative_authors'].split(', '):
        train_iter.append([paper_id, int(negative), 0])

val_iter = []
for index, row in val_data.iterrows():
    paper_id = row['paper_id']
    for author in row['citation_authors'].split(', '):
        val_iter.append([paper_id, int(author), 1])
    for negative in row['negative_authors'].split(', '):
        val_iter.append([paper_id, int(negative), 0])

test_iter = []
for index, row in test_data.iterrows():
    paper_id = row['paper_id']
    for author in row['citation_authors'].split(', '):
        test_iter.append([paper_id, int(author), 1])
    for negative in row['negative_authors'].split(', '):
        test_iter.append([paper_id, int(negative), 0])

In [128]:
# data loader

class PairDataset(Dataset):
  def __init__(self, x):
    self.x_paper = x[:,0]
    self.x_author = x[:,1]
    self.y = x[:,2]
  
  def __len__(self):
    return len(self.x_paper)

  def __getitem__(self, idx):
    x1_sample = df_abstract['abstract'][self.x_paper[idx]] # abstract of paper
    x2_sample = df_authors.loc[df_authors['author_id'] == x_author[idx], 'paper_ids']
    x2_sample = str(x2_sampl.values[0]).split(', ')
    x2_sampl = [int(x) for x in x2_sampl] # list of paper ids
    y_sample = self.y[idx]
    
    return x1_sample, x2_sample, y_sample

train_data = PairDataset(torch.tensor(train_iter))
valid_data = PairDataset(torch.tensor(val_iter))
test_data = PairDataset(torch.tensor(test_iter))

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [124]:
# model

537037

In [ ]:
# training loop

for epoch in range(epochs):
    model.train()
    for paper_id, author_id_ls, label in train_loader:
        optimizer.zero_grad()
        